In [21]:
%%time

import requests
from bs4 import BeautifulSoup
import pandas as pd
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import re


start_year = 2005
end_year = 2021

Years = range(start_year,end_year+1)

Countries = ['Portugal','Spain','England','Italy','Germany','France']

CPU times: user 23 µs, sys: 1 µs, total: 24 µs
Wall time: 27.9 µs


In [22]:
table_fifa_all_years = pd.read_excel('Table_fifa_2005_2021.xlsx')
table_fifa_all_years = table_fifa_all_years.drop(['Unnamed: 0'], axis = 1)
table_fifa_all_years

,Year,Country,Name,ATT,MID,DEF,OVR,Link-team,Team-ID,Fifa_team_all_names,Rival_team,Budget_Mill_€
0,2005,Portugal,FC Porto,85,82,77,81,/team/236/fc-porto/fifa05/,236,"['FC Porto', 'F.C. Porto']",Benfica,16.0
1,2005,Portugal,Benfica,80,74,74,75,/team/234/benfica/fifa05/,234,"['Benfica', 'Sport Lisboa Benfica', 'SL Benfica']",Sporting Lisbon,5.0
2,2005,Portugal,Sporting Lisbon,63,70,65,68,/team/237/sporting-lisbon/fifa05/,237,"['Sporting Lisbon', 'Sporting CP Lisbon', 'Spo...",Benfica,1.7
3,2005,Portugal,Belenenses,67,62,62,64,/team/1889/belenenses/fifa05/,1889,"['Belenenses', 'CF Os Belenenses', 'Belém', 'C...",Boavista,0.4
4,2005,Portugal,Nacional,69,59,61,63,/team/1891/nacional/fifa05/,1891,"['Nacional', 'Clube Desportivo Nacional', 'CD ...",Marítimo,0.5
...,...,...,...,...,...,...,...,...,...,...,...,...
1951,2021,France,FC Lorient,71,71,70,71,/team/217/fc-lorient/fifa21/,217,"['Lorient', 'FC Lorient Bretagne Sud', 'FC Lor...",Stade Rennais FC,5.0
1952,2012,England,Arsenal,84,80,81,82,/team/1/arsenal/fifa12/,1,"['Arsenal', 'Arsenal FC']",Tottenham Hotspur,30.0
1953,2013,England,Southampton,73,74,71,73,/team/17/southampton/fifa13/,17,['Southampton'],Portsmouth,7.5
1954,2014,Spain,Osasuna,71,71,73,72,/team/479/osasuna/fifa14/,479,"['Osasuna', 'Club Atlético Osasuna', 'CA Osasu...",Athletic Club de Bilbao,4.8


In [23]:
table_all_years = pd.read_excel('All_teams_results_leagues_countries_2005_2021_edited.xlsx')
table_all_years = table_all_years.drop(['Unnamed: 0'], axis = 1)
table_all_years.head(3)

,Year,Country,Date,Datetime_date_list,Competition_original_name_URL,Competition,Home_team,Away_team,Result,Home_score,Away_score,1x2,Points_Home_Team,Points_Away_Team,number_of_games_last_days_home_team,number_of_games_last_days_away_team
0,2005,International,24 Feb 05,2005-02-24,Europa League,Europa League,Benfica,CSKA Moskva,1 - 1,1,1,x,1,1,4,1
1,2005,International,17 Feb 05,2005-02-17,Europa League,Europa League,CSKA Moskva,Benfica,2 - 0,2,0,1,3,0,0,3
2,2005,International,02 Dec 04,2004-12-02,Europa League,Europa League,KSK Beveren,Benfica,0 - 3,0,3,2,0,3,0,4


In [24]:
%%time

# create a dictionary whose keys are the team names in the original table
# and the values are the possible names in FIFA for each one of those teams

d = {}

# Build 2 lists: one with unique values of all teams in original table and
# another with unique values of all teams in fifa table

df1 = table_all_years[table_all_years['Competition']=='National League']

for year in Years:
    for country in Countries:
        original_teams_list_year_country = list(set(
            list(df1[(df1['Year'] == year) & 
                     (df1['Country'] == country)]
                     ['Home_team']) +\
            list(df1[(df1['Year'] == year) & 
                     (df1['Country'] == country)]
                     ['Away_team'])))

        fifa_teams_list_year_country = list(set(
            list(table_fifa_all_years[(table_fifa_all_years['Year'] == year) & 
                     (table_fifa_all_years['Country'] == country)]
                     ['Name'])))
        
        print(year)
        print(country)
        print('\n')

        

# Code to find a possible mismatch between the number of teams per year and
# per country in the original and fifa tables (of data is accurate, it should
# not happen)
        

        
        if len(original_teams_list_year_country) != len(fifa_teams_list_year_country):
            print('********************************************************')
            print('********************************************************')
            print('********************************************************')
            print(f'ERROR: {year},{country},\n\
            Number of teams original table: {len(original_teams_list_year_country)},\n\
            Number of teams fifa table: {len(fifa_teams_list_year_country)}')
            print('********************************************************')
            print('********************************************************')
            print('********************************************************')
            print('\n')
        
        else:

            # We create an auxiliar DataFrame (df) in order to sort the list of original
            # teams per year and per country by wuzzyfuzzy score, starting on the highest
            # score. This will allow to start matching names that have a higher level of
            # confidence, and consequently will allow to have a shorter list of names to
            # match for the last teams, whose level of confidence is lower.
            
                
            data = {'Original_name':original_teams_list_year_country}
            df = pd.DataFrame(data)
            
            df['Fifa_name_suggested'] = df['Original_name'].apply(
        (lambda x: process.extractOne(x, fifa_teams_list_year_country)[0])
    )
            
            df['Fifa_name_suggested_fuzzywuzzy_score'] = df['Original_name'].apply(
        (lambda x: process.extractOne(x, fifa_teams_list_year_country)[1])
    )
            
            df.sort_values(by = ['Fifa_name_suggested_fuzzywuzzy_score'], ascending = False,
                          inplace = True)
            
            original_teams_list_year_country = list(df['Original_name'])
            
  

            # Here we look for the best match between an original team name and a fifa team
            # name. For that we run different loops with different levels of fuzzywuzzy
            # score.
            
            
            fuzzywuzzy_score_level = [100,90,80,70,60,50,40,30,20,10,0]
            
            for original_team in original_teams_list_year_country:
                
                if original_team in d.keys():
                    continue
                
                else:

                    for score_level in fuzzywuzzy_score_level:

                        fuzzywuzzy_min_score = score_level

                        fifa_team_suggested = process.extractOne(original_team, 
                                fifa_teams_list_year_country)[0]
                        fifa_team_suggested_score = process.extractOne(original_team, 
                                fifa_teams_list_year_country)[1]


                        if fifa_team_suggested_score >= fuzzywuzzy_min_score:
                            fifa_teams_list_year_country.remove(fifa_team_suggested)

                            if fifa_team_suggested not in d.values():
                                
                                
                                if original_team not in d.keys():
                                    d[original_team] = table_fifa_all_years.loc[table_fifa_all_years['Name']==fifa_team_suggested]['Fifa_team_all_names'].iloc[0]

                                else:
                                    d[original_team].append(
                                table_fifa_all_years.loc[table_fifa_all_years['Name']==fifa_team_suggested]
                                     ['Fifa_team_all_names']).iloc[0]
                                

                            print(original_team,'-->',fifa_team_suggested,'-->',fifa_team_suggested_score)

                            break
            
            print('\n')
            print('-----------------------------')
            print('\n')

2005
Portugal


Nacional --> Nacional --> 100
Marítimo --> Marítimo --> 100
Rio Ave --> Rio Ave --> 100
Benfica --> Benfica --> 100
Gil Vicente --> Gil Vicente --> 100
Boavista --> Boavista --> 100
Sporting Braga --> Sporting Braga --> 100
Moreirense --> Moreirense --> 100
Beira Mar SC --> SC Beira-Mar --> 95
União de Leiria --> União Leiria --> 95
Os Belenenses --> Belenenses --> 95
Porto --> FC Porto --> 90
Estoril --> Estoril Praia --> 90
Penafiel --> F.C. Penafiel --> 90
Vitória Guimarães --> Vitória SC --> 86
Vitória Setúbal --> Vitória Futebol Clube --> 86
Sporting CP --> Sporting Lisbon --> 80
Académica --> Coimbra --> 40


-----------------------------


2005
Spain


Osasuna --> Osasuna --> 100
Villarreal --> Villarreal --> 100
Valencia --> Valencia --> 100
Albacete --> Albacete --> 100
Sevilla --> Sevilla --> 100
Real Betis --> Real Betis --> 100
Real Madrid --> Real Madrid --> 100
Numancia --> Numancia --> 100
Levante --> Levante --> 100
Real Zaragoza --> Real Zaragoza --> 10

Novara --> Novara --> 100


-----------------------------


2012
Germany


FC Augsburg --> FC Augsburg --> 100


-----------------------------


2012
France


Dijon FCO --> Dijon FCO --> 100
Evian Thonon Gaillard --> Evian Thonon Gaillard FC --> 95


-----------------------------


2013
Portugal




-----------------------------


2013
Spain




-----------------------------


2013
England




-----------------------------


2013
Italy


Pescara --> Pescara --> 100


-----------------------------


2013
Germany


Fortuna Düsseldorf --> Fortuna Düsseldorf --> 100
Greuther Fürth --> SpVgg Greuther Fürth --> 95


-----------------------------


2013
France


Stade de Reims --> Stade de Reims --> 100


-----------------------------


2014
Portugal


Arouca --> Arouca --> 100


-----------------------------


2014
Spain


Elche --> Elche Club de Fútbol --> 90


-----------------------------


2014
England


Cardiff City --> Cardiff City --> 100


-----------------------------


2014
Italy



In [25]:
d

{'Nacional': "['Nacional', 'Clube Desportivo Nacional', 'CD Nacional', 'Funchal']",
 'Marítimo': "['Marítimo', 'Marítimo da Madeira', 'CS Marítimo', 'C. Funchal']",
 'Rio Ave': "['Rio Ave', 'Rio Ave FC']",
 'Benfica': "['Benfica', 'Sport Lisboa Benfica', 'SL Benfica']",
 'Gil Vicente': "['Gil Vicente', 'V. Barcelos', 'Gil Vicente FC']",
 'Boavista': "['Boavista', 'Boavista Futebol Clube', 'Boavista FC']",
 'Sporting Braga': "['Sporting Braga', 'SC Braga']",
 'Moreirense': "['Moreirense', 'Moreira de Cónegos', 'Moreirense FC']",
 'Beira Mar SC': "['SC Beira-Mar', 'SC Beira Mar']",
 'União de Leiria': "['União Leiria', 'União Desportivo de Leiria', 'União de Leiria, SAD']",
 'Os Belenenses': "['Belenenses', 'CF Os Belenenses', 'Belém', 'C.F. Os Belenenses', 'Os Belenenses']",
 'Porto': "['FC Porto', 'F.C. Porto']",
 'Estoril': "['Estoril Praia', 'Estoril', 'GD Estoril-Praia']",
 'Penafiel': "['F.C. Penafiel', 'FC Penafiel']",
 'Vitória Guimarães': "['Vitória SC', 'Vitória de Guimarães', 

In [26]:
# Manually correct the errors of the dictionary

print('Before manual correction:')
print('Milan:',d['Milan'])
print('Inter:',d['Inter'])
print('Fiorentina:',d['Fiorentina'])
print('Köln:',d['Köln'])
print('Feirense:',d['Feirense'])
print('Saint-Étienne:',d['Saint-Étienne'])
print('PSG:',d['PSG'])

d['Milan'] = table_fifa_all_years[table_fifa_all_years['Name']=='AC Milan']\
['Fifa_team_all_names'].iloc[0]
d['Inter'] = table_fifa_all_years[table_fifa_all_years['Name']=='Inter Milan']\
['Fifa_team_all_names'].iloc[0]
d['Fiorentina'] = table_fifa_all_years[table_fifa_all_years['Name']=='Firenze']\
['Fifa_team_all_names'].iloc[0]
d['Köln'] = table_fifa_all_years[table_fifa_all_years['Name']=='FC Cologne']\
['Fifa_team_all_names'].iloc[0]
d['Feirense'] = table_fifa_all_years[table_fifa_all_years['Name']=='F. Santa Maria da Feira']\
['Fifa_team_all_names'].iloc[0]
d['Saint-Étienne'] = table_fifa_all_years[table_fifa_all_years['Name']=='AS Saint-Etienne']\
['Fifa_team_all_names'].iloc[0]
d['PSG'] = table_fifa_all_years[table_fifa_all_years['Name']=='Paris Saint-Germain']\
['Fifa_team_all_names'].iloc[0]

print('\n')
print('After manual correction:')
print('Milan:',d['Milan'])
print('Inter:',d['Inter'])
print('Fiorentina:',d['Fiorentina'])
print('Köln:',d['Köln'])
print('Feirense:',d['Feirense'])
print('Saint-Étienne:',d['Saint-Étienne'])
print('PSG:',d['PSG'])

Before manual correction:
Milan: ['AC Milan', 'Milan']
Inter: ['Inter Milan', 'Inter']
Fiorentina: ['Firenze', 'Fiorentina', 'ACF Fiorentina']
Köln: ['Hertha BSC Berlin', 'Hertha BSC', 'Hertha Berlin']
Feirense: ['Vitória Futebol Clube', 'Vitória de Setúbal', 'Vitória Setúbal']
Saint-Étienne: ['Paris Saint-Germain']
PSG: ['AS Saint-Etienne', 'A.S. Saint-Etienne', 'AS Saint-Étienne']


After manual correction:
Milan: ['AC Milan', 'Milan']
Inter: ['Inter Milan', 'Inter']
Fiorentina: ['Firenze', 'Fiorentina', 'ACF Fiorentina']
Köln: ['FC Cologne', '1. FC Köln']
Feirense: ['F. Santa Maria da Feira', 'CD Feirense']
Saint-Étienne: ['AS Saint-Etienne', 'A.S. Saint-Etienne', 'AS Saint-Étienne']
PSG: ['Paris Saint-Germain']


In [27]:
d

{'Nacional': "['Nacional', 'Clube Desportivo Nacional', 'CD Nacional', 'Funchal']",
 'Marítimo': "['Marítimo', 'Marítimo da Madeira', 'CS Marítimo', 'C. Funchal']",
 'Rio Ave': "['Rio Ave', 'Rio Ave FC']",
 'Benfica': "['Benfica', 'Sport Lisboa Benfica', 'SL Benfica']",
 'Gil Vicente': "['Gil Vicente', 'V. Barcelos', 'Gil Vicente FC']",
 'Boavista': "['Boavista', 'Boavista Futebol Clube', 'Boavista FC']",
 'Sporting Braga': "['Sporting Braga', 'SC Braga']",
 'Moreirense': "['Moreirense', 'Moreira de Cónegos', 'Moreirense FC']",
 'Beira Mar SC': "['SC Beira-Mar', 'SC Beira Mar']",
 'União de Leiria': "['União Leiria', 'União Desportivo de Leiria', 'União de Leiria, SAD']",
 'Os Belenenses': "['Belenenses', 'CF Os Belenenses', 'Belém', 'C.F. Os Belenenses', 'Os Belenenses']",
 'Porto': "['FC Porto', 'F.C. Porto']",
 'Estoril': "['Estoril Praia', 'Estoril', 'GD Estoril-Praia']",
 'Penafiel': "['F.C. Penafiel', 'FC Penafiel']",
 'Vitória Guimarães': "['Vitória SC', 'Vitória de Guimarães', 

In [28]:
df1['Home_team_fifa_team_all_names'] = df1['Home_team'].apply(
        (lambda x: d[x])
    )

df1['Away_team_fifa_team_all_names'] = df1['Away_team'].apply(
        (lambda x: d[x])
    )

df1

<ipython-input-28-465b3e920141>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['Home_team_fifa_team_all_names'] = df1['Home_team'].apply(
<ipython-input-28-465b3e920141>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['Away_team_fifa_team_all_names'] = df1['Away_team'].apply(


,Year,Country,Date,Datetime_date_list,Competition_original_name_URL,Competition,Home_team,Away_team,Result,Home_score,Away_score,1x2,Points_Home_Team,Points_Away_Team,number_of_games_last_days_home_team,number_of_games_last_days_away_team,Home_team_fifa_team_all_names,Away_team_fifa_team_all_names
8,2005,Portugal,22 May 05,2005-05-22,Liga Portuguesa,National League,Boavista,Benfica,1 - 1,1,1,x,1,1,2,2,"['Boavista', 'Boavista Futebol Clube', 'Boavis...","['Benfica', 'Sport Lisboa Benfica', 'SL Benfica']"
9,2005,Portugal,14 May 05,2005-05-14,Liga Portuguesa,National League,Benfica,Sporting CP,1 - 0,1,0,1,3,0,3,5,"['Benfica', 'Sport Lisboa Benfica', 'SL Benfica']","['Sporting Lisbon', 'Sporting CP Lisbon', 'Spo..."
10,2005,Portugal,07 May 05,2005-05-07,Liga Portuguesa,National League,Penafiel,Benfica,1 - 0,1,0,1,3,0,3,4,"['F.C. Penafiel', 'FC Penafiel']","['Benfica', 'Sport Lisboa Benfica', 'SL Benfica']"
11,2005,Portugal,30 Apr 05,2005-04-30,Liga Portuguesa,National League,Benfica,Os Belenenses,1 - 0,1,0,1,3,0,4,3,"['Benfica', 'Sport Lisboa Benfica', 'SL Benfica']","['Belenenses', 'CF Os Belenenses', 'Belém', 'C..."
12,2005,Portugal,24 Apr 05,2005-04-24,Liga Portuguesa,National League,Estoril,Benfica,1 - 2,1,2,2,0,3,3,4,"['Estoril Praia', 'Estoril', 'GD Estoril-Praia']","['Benfica', 'Sport Lisboa Benfica', 'SL Benfica']"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52385,2021,France,30 Aug 20,2020-08-30,Ligue 1,National League,Nantes,Nîmes,2 - 1,2,1,1,3,0,1,1,['FC Nantes'],['Nîmes Olympique']
52389,2021,Friendly match,30 May 21,2021-05-30,Ligue 1 - Play Offs Ascenso,National League,Nantes,Toulouse,0 - 1,0,1,2,0,3,3,1,['FC Nantes'],"['Toulouse', 'Toulouse F.C.', 'Toulouse FC', '..."
52390,2021,Friendly match,27 May 21,2021-05-27,Ligue 1 - Play Offs Ascenso,National League,Toulouse,Nantes,1 - 2,1,2,2,0,3,0,3,"['Toulouse', 'Toulouse F.C.', 'Toulouse FC', '...",['FC Nantes']
52391,2021,France,14 Feb 21,2021-02-14,Ligue 1,National League,Dijon FCO,Nîmes,0 - 2,0,2,2,0,3,6,4,['Dijon FCO'],['Nîmes Olympique']


In [42]:
Year = 2005
Country = 'Portugal'
Home_team_fifa_team_all_names = str(['F.C. Penafiel', 'FC Penafiel'])

table_fifa_all_years.loc[(table_fifa_all_years['Year'] == Year) &
                     (table_fifa_all_years['Country'] == Country) &
                     (table_fifa_all_years['Fifa_team_all_names'] == Home_team_fifa_team_all_names)
                    ]

,Year,Country,Name,ATT,MID,DEF,OVR,Link-team,Team-ID,Fifa_team_all_names,Rival_team,Budget_Mill_€
6,2005,Portugal,F.C. Penafiel,64,67,55,61,/team/100741/f-c-penafiel/fifa05/,100741,"['F.C. Penafiel', 'FC Penafiel']",FC Porto,0.1


In [50]:
columns = ['ATT','MID','DEF','OVR','Rival_team','Budget_Mill_€']

Year = 2005
Country = 'Portugal'
Home_team_fifa_team_all_names = str(['F.C. Penafiel', 'FC Penafiel'])

for column in columns:
    print(column,'-->',table_fifa_all_years.loc[(table_fifa_all_years['Year'] == Year) &
                         (table_fifa_all_years['Country'] == Country) &
                         (table_fifa_all_years['Fifa_team_all_names'] == Home_team_fifa_team_all_names)
                        ][column].iloc[0])

ATT --> 64
MID --> 67
DEF --> 55
OVR --> 61
Rival_team --> FC Porto
Budget_Mill_€ --> 0.1
